# Twilio Voice API - Answering Machine Detection Examples

This notebook demonstrates various **Answering Machine Detection (AMD)** scenarios using Twilio's **Voice API**.

### Prerequisites

- Twilio Account with **Account SID** and **Auth Token**
- Two **Twilio phone numbers** (one for *outbound calls*, one for testing different *incoming call behaviors*)
- Python environment with required libraries

In [2]:
# Install required libraries
%pip install twilio flask python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Start a simple Flask server for webhook handling (run in separate terminal if needed)
from flask import Flask, request
from twilio.twiml.voice_response import VoiceResponse
import threading
import time

app = Flask(__name__)

@app.route("/webhook", methods=['POST'])
def handle_webhook():
    """Handle Twilio webhooks for call status updates"""
    response = VoiceResponse()
    
    # Log the webhook data
    print("Webhook received:")
    for key, value in request.form.items():
        print(f"  {key}: {value}")
    
    return str(response)

def run_server():
    app.run(host='0.0.0.0', port=3000, debug=False)

# Start server in background thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(2)
print("Webhook server started on http://localhost:5000")

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:3000
 * Running on http://192.168.15.164:3000
Press CTRL+C to quit


Webhook server started on http://localhost:5000


127.0.0.1 - - [09/Jun/2025 13:05:04] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [09/Jun/2025 13:05:04] "GET /favicon.ico HTTP/1.1" 404 -


In [4]:
import os
from twilio.rest import Client
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Twilio client with environment variables
ACCOUNT_SID = os.getenv('TWILIO_ACCOUNT_SID')
AUTH_TOKEN = os.getenv('TWILIO_AUTH_TOKEN')
AMD_TWILIO_NUMBER = os.getenv('TWILIO_PHONE_NUMBER')
INCOMING_NUMBER = os.getenv('INCOMING_PHONE_NUMBER')

if not ACCOUNT_SID or not AUTH_TOKEN:
    print("Please set TWILIO_ACCOUNT_SID and TWILIO_AUTH_TOKEN environment variables")
    print("  You can create a .env file with:")
    print("  TWILIO_ACCOUNT_SID=your_account_sid_here")
    print("  TWILIO_AUTH_TOKEN=your_auth_token_here")
else:
    client = Client(ACCOUNT_SID, AUTH_TOKEN)
    print("Twilio client initialized successfully")
    print(f"Account SID: {ACCOUNT_SID[:8]}...")

Twilio client initialized successfully
Account SID: ACdf269d...


In [5]:
# Define AMD use case configurations
AMD_USE_CASES = {
    "1": {
        "name": "Machine Detected - Default Settings",
        "description": "Standard AMD with default sensitivity",
        "amd": "true",
        "amd_status_callback": "http://localhost:5000/webhook",
        "machine_detection_timeout": 10,
        "machine_detection_speech_threshold": 2400,
        "machine_detection_speech_end_threshold": 1200,
        "machine_detection_silence_timeout": 5000,
        "callReason": "machine-detected"
    },
    "2": {
        "name": "Human Detected - High Sensitivity",
        "description": "AMD configured to be more likely to detect humans",
        "amd": "true",
        "amd_status_callback": "http://localhost:5000/webhook",
        "machine_detection_timeout": 15,
        "machine_detection_speech_threshold": 1800,
        "machine_detection_speech_end_threshold": 800,
        "machine_detection_silence_timeout": 3000,
        "callReason": "human-detected"
    },
    "3": {
        "name": "Long Message Detection",
        "description": "Optimized for detecting long answering machine messages",
        "amd": "true",
        "amd_status_callback": "http://localhost:5000/webhook",
        "machine_detection_timeout": 20,
        "machine_detection_speech_threshold": 3000,
        "machine_detection_speech_end_threshold": 1500,
        "machine_detection_silence_timeout": 7000,
        "callReason": "long-message-detected"
    },
    "4": {
        "name": "Short Message Detection",
        "description": "Optimized for detecting short answering machine messages",
        "amd": "true",
        "amd_status_callback": "http://localhost:5000/webhook",
        "machine_detection_timeout": 8,
        "machine_detection_speech_threshold": 1500,
        "machine_detection_speech_end_threshold": 600,
        "machine_detection_silence_timeout": 3000,
        "callReason": "short-message-detected"
    },
    "5": {
        "name": "Recorded Message Handling",
        "description": "AMD with TwiML for handling recorded messages",
        "amd": "true",
        "amd_status_callback": "http://localhost:5000/webhook",
        "machine_detection_timeout": 12,
        "url": "http://localhost:5000/handle_amd",
        "callReason": "recorded-message-handling"
    }
}

print("Twilio AMD Use Cases:")
print("=" * 50)
for key, case in AMD_USE_CASES.items():
    print(f"{key}. {case['name']}")
    print(f"   {case['description']}")
    print()

Twilio AMD Use Cases:
1. Machine Detected - Default Settings
   Standard AMD with default sensitivity

2. Human Detected - High Sensitivity
   AMD configured to be more likely to detect humans

3. Long Message Detection
   Optimized for detecting long answering machine messages

4. Short Message Detection
   Optimized for detecting short answering machine messages

5. Recorded Message Handling
   AMD with TwiML for handling recorded messages



In [6]:
def make_amd_call(from_number, to_number, use_case_id):
    """
    Make an outbound call with specific AMD configuration
    
    Args:
        from_number (str): Your Twilio phone number
        to_number (str): Destination phone number
        use_case_id (str): ID of the AMD use case to apply
    """
    
    if use_case_id not in AMD_USE_CASES:
        print(f"Invalid use case ID: {use_case_id}")
        return None
    
    use_case = AMD_USE_CASES[use_case_id]
    print(f"Making call with: {use_case['name']}")
    print(f"Description: {use_case['description']}")
    
    try:
        # Prepare call parameters
        call_params = {
            'to': to_number,
            'from_': from_number,
            'machine_detection': use_case.get('amd', 'false'),
            'status_callback': use_case.get('amd_status_callback'),
            'status_callback_event': ['initiated', 'ringing', 'answered', 'completed'],
            'call_reason': use_case.get('callReason', 'unknown')
        }
        
        # Add AMD-specific parameters
        amd_params = [
            'machine_detection_timeout',
            'machine_detection_speech_threshold', 
            'machine_detection_speech_end_threshold',
            'machine_detection_silence_timeout'
        ]
        
        for param in amd_params:
            if param in use_case:
                call_params[param] = use_case[param]
        
        # Set TwiML URL or default message
        if 'url' in use_case:
            call_params['url'] = use_case['url']
        else:
            # Default TwiML for testing
            call_params['twiml'] = '''
            <Response>
                <Say voice="alice">
                    Hello! This is a test call from Twilio with Answering Machine Detection. 
                    If you're hearing this, the system detected a human answered the phone.
                    Thank you for testing!
                </Say>
                <Pause length="2"/>
                <Say voice="alice">Goodbye!</Say>
            </Response>
            '''
        
        # Make the call
        call = client.calls.create(**call_params)
        
        print(f"Call initiated successfully!")
        print(f"Call SID: {call.sid}")
        print(f"Status: {call.status}")
        print(f"From: {call._from}")
        print(f"To: {call.to}")
        
        return call
    
    except Exception as e:
        print(f"Error making call: {str(e)}")
        return None

# Example usage - UPDATE THESE PHONE NUMBERS
FROM_NUMBER = AMD_TWILIO_NUMBER  # Your Twilio phone number
TO_NUMBER = INCOMING_NUMBER    # Destination phone number (another Twilio number for testing)

print("Phone Numbers Configuration:")
print(f"From: {FROM_NUMBER}")
print(f"To: {TO_NUMBER}")
print("\nMake sure to update the phone numbers above before running calls!")
print("\n" + "="*60)

# Make a test call with default AMD settings
if FROM_NUMBER != "+1234567890" and TO_NUMBER != "+1987654321":
    call = make_amd_call(FROM_NUMBER, TO_NUMBER, "1")
else:
    print("Please update FROM_NUMBER and TO_NUMBER variables with your actual Twilio phone numbers")

Phone Numbers Configuration:
From: +551150266619
To: +551148583195

Make sure to update the phone numbers above before running calls!

Making call with: Machine Detected - Default Settings
Description: Standard AMD with default sensitivity
Call initiated successfully!
Call SID: CAd349be00c62dbf79b0a934118e2bece6
Status: queued
From: +551150266619
To: +551148583195


In [7]:
def interactive_amd_test():
    """Interactive function to test different AMD scenarios"""
    
    print("Interactive AMD Testing")
    print("=" * 40)
    
    # Get phone numbers
    print("Please enter your phone numbers for testing:")
    use_env = input("Use environment variables for phone numbers? (yes/no): ").strip().lower()
    if use_env == "yes":
        from_num = AMD_TWILIO_NUMBER
        to_num = INCOMING_NUMBER
    else:   
        # Prompt for phone numbers
        from_num = input("Enter your Twilio phone number (e.g., +1234567890): ").strip()
        to_num = input("Enter destination phone number (e.g., +1987654321): ").strip()
    
    if not from_num or not to_num:
        print("Phone numbers are required!")
        return
    
    while True:
        print("\nSelect AMD Use Case:")
        print("-" * 30)
        for key, case in AMD_USE_CASES.items():
            print(f"{key}. {case['name']}")
        print("0. Exit")
        
        choice = input("\nEnter your choice (0-5): ").strip()
        
        if choice == "0":
            print("Goodbye!")
            break
        elif choice in AMD_USE_CASES:
            print(f"\nExecuting: {AMD_USE_CASES[choice]['name']}")
            call = make_amd_call(from_num, to_num, choice)
            
            if call:
                print(f"\nCall Details:")
                print(f"   SID: {call.sid}")
                print(f"   Status: {call.status}")
                
                # Wait a moment and check status
                import time
                time.sleep(3)
                updated_call = client.calls(call.sid).fetch()
                print(f"   Updated Status: {updated_call.status}")
                
                if hasattr(updated_call, 'answering_machine_detection'):
                    print(f"   AMD Result: {updated_call.answering_machine_detection}")
        else:
            print("Invalid choice! Please select 0-5.")

# Run interactive test
interactive_amd_test()
# Uncomment the line above to run interactive AMD testing

Interactive AMD Testing
Please enter your phone numbers for testing:

Select AMD Use Case:
------------------------------
1. Machine Detected - Default Settings
2. Human Detected - High Sensitivity
3. Long Message Detection
4. Short Message Detection
5. Recorded Message Handling
0. Exit

Executing: Machine Detected - Default Settings
Making call with: Machine Detected - Default Settings
Description: Standard AMD with default sensitivity
Call initiated successfully!
Call SID: CAc8eb29867723efb368925642909944ea
Status: queued
From: +551150266619
To: +551148583195

Call Details:
   SID: CAc8eb29867723efb368925642909944ea
   Status: queued
   Updated Status: busy

Select AMD Use Case:
------------------------------
1. Machine Detected - Default Settings
2. Human Detected - High Sensitivity
3. Long Message Detection
4. Short Message Detection
5. Recorded Message Handling
0. Exit

Executing: Human Detected - High Sensitivity
Making call with: Human Detected - High Sensitivity
Description: AM

In [8]:
def monitor_call_status(call_sid, max_wait_time=60):
    """
    Monitor a call's status and AMD results
    
    Args:
        call_sid (str): The SID of the call to monitor
        max_wait_time (int): Maximum time to wait in seconds
    """
    import time
    
    print(f"Monitoring call: {call_sid}")
    print("-" * 50)
    
    start_time = time.time()
    last_status = None
    
    while (time.time() - start_time) < max_wait_time:
        try:
            call = client.calls(call_sid).fetch()
            
            if call.status != last_status:
                print(f"{time.strftime('%H:%M:%S')} - Status: {call.status}")
                
                # Check for AMD results
                if hasattr(call, 'answering_machine_detection'):
                    amd_result = call.answering_machine_detection
                    if amd_result and amd_result != 'unknown':
                        print(f"AMD Result: {amd_result}")
                
                last_status = call.status
                
                if call.status in ['completed', 'failed', 'canceled', 'busy', 'no-answer']:
                    print(f"Call ended with status: {call.status}")
                    if call.duration:
                        print(f"Duration: {call.duration} seconds")
                    break
            
            time.sleep(2)
            
        except Exception as e:
            print(f"Error monitoring call: {str(e)}")
            break

    print("Monitoring complete")

# Example: Monitor the last call made
# Replace 'CALL_SID_HERE' with an actual call SID
# monitor_call_status('CALL_SID_HERE')
print("Use monitor_call_status('YOUR_CALL_SID') to monitor a specific call")

Use monitor_call_status('YOUR_CALL_SID') to monitor a specific call


In [11]:
def analyze_recent_calls(limit=10):
    """Analyze recent calls and their AMD results"""
    
    print("Recent Calls AMD Analysis")
    print("=" * 50)
    
    try:
        calls = client.calls.list(limit=limit)
        
        amd_stats = {
            'human': 0,
            'machine': 0,
            'fax': 0,
            'unknown': 0,
            'no_amd': 0
        }
        
        print(f"{'Call SID':<35} {'Status':<12} {'AMD Result':<12} {'Duration'}")
        print("-" * 75)
        
        for call in calls:
            duration = call.duration if call.duration else 'N/A'
            amd_result = getattr(call, 'answering_machine_detection', None) or 'no_amd'
            
            print(f"{call.sid:<35} {call.status:<12} {amd_result:<12} {duration}")
            
            # Update stats
            if amd_result in amd_stats:
                amd_stats[amd_result] += 1
            else:
                amd_stats['unknown'] += 1
        
        print("\nAMD Statistics:")
        print("-" * 30)
        for result, count in amd_stats.items():
            if count > 0:
                percentage = (count / len(calls)) * 100
                print(f"{result.title():<12}: {count:>3} ({percentage:.1f}%)")
                
    except Exception as e:
        print(f"Error analyzing calls: {str(e)}")

# Run analysis
analyze_recent_calls()

Recent Calls AMD Analysis
Call SID                            Status       AMD Result   Duration
---------------------------------------------------------------------------
CA7331190f16384e171318902a656cdf64  completed    no_amd       10
CA2c8daef5d04e88e9db41f39415ea375b  completed    no_amd       9
CA359df1ab8f235d8a1c739e4904ea4eaf  completed    no_amd       9
CAbd5167c46b69705faa945127afe8517c  completed    no_amd       9
CAb2e3439a9eb022360e7cf46eba108788  completed    no_amd       9
CA6b793cbb99ee9b0d360d0503c333e8ae  completed    no_amd       9
CA516aea3c0b7f5f016498549c3ee63f9b  busy         no_amd       0
CAe7d366ba31e1591641a9bfce1efb12a4  busy         no_amd       0
CA5e61b681332eb1f642daf098f27ae35b  busy         no_amd       0
CAc8eb29867723efb368925642909944ea  busy         no_amd       0

AMD Statistics:
------------------------------
No_Amd      :  10 (100.0%)


In [13]:
# Advanced TwiML handler for recorded message scenarios
try:
    app
except NameError:
    from flask import Flask
    app = Flask(__name__)

if "handle_amd" not in app.view_functions:
    @app.route("/handle_amd", methods=['POST'])
    def handle_amd():
        """Handle calls with AMD results and provide appropriate TwiML"""
        
        response = VoiceResponse()
        
        # Get AMD result from Twilio
        amd_result = request.form.get('AnsweringMachineDetection', 'unknown')
        
        print(f"AMD Handler called - Result: {amd_result}")
        
        if amd_result == 'machine':
            # Answering machine detected - leave a message
            response.pause(length=1)  # Wait for beep
            response.say(
                "Hello! This is an automated message from Twilio. "
                "We detected an answering machine. This is a test of our "
                "Answering Machine Detection system. Thank you!",
                voice='alice'
            )
            
        elif amd_result == 'human':
            # Human detected - normal conversation
            response.say(
                "Hello! Thank you for answering. This is a test call "
                "from Twilio demonstrating our Answering Machine Detection. "
                "Have a great day!",
                voice='alice'
            )
            
        elif amd_result == 'fax':
            # Fax machine detected
            response.say("Fax machine detected. Hanging up.", voice='alice')
            response.hangup()
            
        else:
            # Unknown or no AMD result
            response.say(
                "Hello! This is a test call from Twilio. "
                "We couldn't determine if this was answered by a human or machine.",
                voice='alice'
            )
        
        return str(response)

print("Advanced TwiML handler configured")
print("   Webhook URL: http://localhost:5000/handle_amd")
print("   Use this URL in AMD use case #5 for advanced message handling")

Advanced TwiML handler configured
   Webhook URL: http://localhost:5000/handle_amd
   Use this URL in AMD use case #5 for advanced message handling
